In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

POPULATION_SIZE = 100
GENERATIONS = 100
CROSSOVER_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.2
MAX_RUNS = 30

def algorithm(data):
    pass


def main():
    folder_path = 'ACI_Project2_2324_Data'
    file_list = os.listdir(folder_path)
    data_dictionary = {}
    for file_name in file_list:
        if file_name.endswith('.csv'):
            # extract the base name of the file (without the extension)
            df_name = os.path.splitext(file_name)[0]
            # construct the full file path
            file_path = os.path.join(folder_path, file_name)
            
            # read the CSV file into a DataFrame with the base name as the variable name
            data_dictionary[df_name] = pd.read_csv(file_path, sep=';', usecols=['Date', 'Close'])


if __name__ == "__main__":
    main()

            Date      Close
0     27/09/2005  19.299999
1     28/09/2005  20.500000
2     29/09/2005  20.209999
3     30/09/2005  21.010000
4     03/10/2005  21.500000
...          ...        ...
4518  11/09/2023  14.090000
4519  12/09/2023  14.110000
4520  13/09/2023  13.310000
4521  14/09/2023  13.260000
4522  15/09/2023  13.310000

[4523 rows x 2 columns]
             Date       Close
0      12/12/1980    0.128348
1      15/12/1980    0.121652
2      16/12/1980    0.112723
3      17/12/1980    0.115513
4      18/12/1980    0.118862
...           ...         ...
10775  11/09/2023  179.360001
10776  12/09/2023  176.300003
10777  13/09/2023  174.210007
10778  14/09/2023  175.740005
10779  15/09/2023  175.009995

[10780 rows x 2 columns]
            Date       Close
0     15/05/1997    0.097917
1     16/05/1997    0.086458
2     19/05/1997    0.085417
3     20/05/1997    0.081771
4     21/05/1997    0.071354
...          ...         ...
6623  11/09/2023  143.100006
6624  12/09/2023  141